In [ ]:
import os
import time
import urllib.parse
import requests
from bs4 import BeautifulSoup

In [ ]:
session = requests.Session()
url = 'https://pip.moi.gov.tw/Publicize/Info/E4020'

# （1）先 GET 頁面取得 token
resp = session.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'html.parser')
token = soup.find('input', {'name': '__RequestVerificationToken'})['value']

In [ ]:
# 2. 要跑的縣市代號與季度
cities = [
    {"臺北市": "A"}, {"新北市": "F"}, {"桃園市": "H"}, {"新竹市": "O"}, {"新竹縣": "J"}, {"臺中市": "B"}, {"臺南市": "D"}, {"高雄市": "E"},
    {"南投縣": "M"}, {"嘉義市": "I"}, {"嘉義縣": "Q"}, {"宜蘭縣": "G"}, {"屏東縣": "T"}, {"彰化縣": "N"}, {"澎湖縣": "X"}, {"臺東縣": "V"},
    {"花蓮縣": "U"}, {"苗栗縣": "K"}, {"連江縣": "Z"}, {"金門縣": "W"}, {"雲林縣": "P"}, {"基隆市":"C"}
]

# cities = [
#     {"臺北市": "A"},
# ]
payload_quarters = [f"{y}Q{q}" for y in range(109, 114) for q in (1,2,3,4)]
file_name_quarters = [f"{y}Y{q}S" for y in range(109, 114) for q in (1,2,3,4)]
# 3. 輸出資料夾
output_dir = r"C:\pylabs\area-risk-flagging\data\pip_moi\district_house_for_tax\raw_data"
os.makedirs(output_dir, exist_ok=True)

In [ ]:
# 4. Base payload
base_payload = {
    # 下列跟 DevTools 裡一模一樣的 key
    "F01Sel":    urllib.parse.quote("住宅供給", encoding="utf-8"),
    "F02Sel":    "TAX02",
    "F01":       urllib.parse.quote("住宅供給", encoding="utf-8"),
    "F02":       "TAX02",
    "ResponseCommand": urllib.parse.quote("檢視查詢結果", encoding="utf-8"),
    "Command":         "exportCSV",
    # 如果 CommandArgument 不需要就留空字串
    "CommandArgument": "",
    "__RequestVerificationToken": token
}

headers = {
    'Referer': url,
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
}

In [ ]:
# 5. 迴圈下載：用 zip 同時取 payload 與檔名用的季度
for city_dict in cities:
    city_name, city_code = next(iter(city_dict.items()))
    for payload_q, file_q in zip(payload_quarters, file_name_quarters):
        payload = base_payload.copy()
        payload["DDATESel"] = payload_q       # 給 POST 的季度
        payload["DDATE"] = payload_q       # 給 POST 的季度
        payload["CitySel"] = city_code       # 給 POST 的縣市代號
        payload["City"] = city_code       # 給 POST 的縣市代號

        r = session.post(url, data=payload, headers=headers)
        r.raise_for_status()

        # 這裡用 file_q 來取代原本的 payload_q
        fname = f"房屋稅籍住宅類_屋齡分_{file_q}_{city_code}_{city_name}.csv"
        path = os.path.join(output_dir, fname)

        with open(path, 'wb') as fp:
            fp.write(r.content)

        print(f"下載完成：{path}")
        time.sleep(0.5)